In [ ]:
# EDA and Modeling: Activity Forecasting using CNN + LSTM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/4522Dev/Dataset/refs/heads/main/dailyActivity_merged.csv')

# Check for missing values and data types
print(df.isna().sum())
print(df.info())

# Changing the datatype of ActivityDate
df["ActivityDate"] = pd.to_datetime(df["ActivityDate"], format="%m/%d/%Y")

# Calculate Total Minutes of activity
df["TotalMinutes"] = (df["VeryActiveMinutes"] +
                      df["FairlyActiveMinutes"] +
                      df["LightlyActiveMinutes"] +
                      df["SedentaryMinutes"])

# Descriptive statistics
print(df.describe())

# Scatter plots
px.scatter(df, x="Calories", y="TotalSteps", size="VeryActiveMinutes", trendline="ols",
           title="Relationship between Calories & Total Steps").show()

px.scatter(df, x="TrackerDistance", y="TotalDistance", trendline="ols",
           title="Relationship between Tracker & Total Distances").show()

# Pie chart: Total Active Minutes
label = ["Very Active Minutes", "Fairly Active Minutes", "Lightly Active Minutes", "Inactive Minutes"]
counts = df[["VeryActiveMinutes", "FairlyActiveMinutes", "LightlyActiveMinutes", "SedentaryMinutes"]].mean()
colors = ['gold', 'lightgreen', "pink", "blue"]

go.Figure(data=[go.Pie(labels=label, values=counts)]) \
  .update_layout(title_text='Total Active Minutes') \
  .update_traces(textinfo='value', marker=dict(colors=colors, line=dict(color='black', width=3))).show()

# Pie chart: Total Active Distance
label = ["Very Active Distances", "Fairly Active Distances", "Lightly Active Distances", "Inactive Distances"]
counts = df[["VeryActiveDistance", "ModeratelyActiveDistance", "LightActiveDistance", "SedentaryActiveDistance"]].mean()
colors = ['gold', 'lightgreen', "pink", "blue"]

go.Figure(data=[go.Pie(labels=label, values=counts)]) \
  .update_layout(title_text='Total Active Distance') \
  .update_traces(textinfo='label+percent', marker=dict(colors=colors, line=dict(color='black', width=3))).show()

# Day extraction
df["Day"] = df["ActivityDate"].dt.day_name()

# Bar charts for minutes
fig = go.Figure()
for activity in ['VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes']:
    fig.add_trace(go.Bar(
        x=df["Day"], y=df[activity], name=activity.replace('Minutes', ''),
        marker_color={'VeryActiveMinutes': 'red', 'FairlyActiveMinutes': 'black', 'LightlyActiveMinutes': 'blue'}[activity]
    ))
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Daily Activity Levels')
fig.show()

# Bar charts for distance
fig = go.Figure()
for activity in ['VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance']:
    fig.add_trace(go.Bar(
        x=df["Day"], y=df[activity], name=activity.replace('Distance', ''),
        marker_color={'VeryActiveDistance': 'red', 'ModeratelyActiveDistance': 'black', 'LightActiveDistance': 'blue'}[activity]
    ))
fig.update_layout(barmode='group', xaxis_tickangle=-45, title='Daily Activity Levels by Distance')
fig.show()

# Custom pie charts
colors_dict = {'Monday': 'red', 'Tuesday': 'green', 'Wednesday': 'blue',
               'Thursday': 'orange', 'Friday': 'purple', 'Saturday': 'brown', 'Sunday': 'pink'}

go.Figure(data=[go.Pie(labels=df["Day"].value_counts().index, values=df["SedentaryMinutes"])]) \
  .update_layout(title_text='Inactive Minutes Daily') \
  .update_traces(textinfo='value', marker=dict(colors=list(colors_dict.values()), line=dict(color='red', width=3))).show()

# Calories per day
calories_per_day = df.groupby("Day")["Calories"].sum()
go.Figure(data=[go.Pie(labels=calories_per_day.index, values=calories_per_day,
                       marker_colors=[colors_dict[d] for d in calories_per_day.index])]) \
  .update_layout(title_text='Calories Burned Daily') \
  .update_traces(textinfo='value', marker=dict(line=dict(color='black', width=3))).show()

# Total distance per day
distances_per_day = df.groupby("Day")["TotalDistance"].sum()
go.Figure(data=[go.Pie(labels=distances_per_day.index, values=distances_per_day,
                       marker_colors=[colors_dict[d] for d in distances_per_day.index])]) \
  .update_layout(title_text='Total Distance Daily') \
  .update_traces(textinfo='value', marker=dict(line=dict(color='black', width=3))).show()

# Total steps per day
steps_per_day = df.groupby("Day")["TotalSteps"].sum()
go.Figure(data=[go.Pie(labels=steps_per_day.index, values=steps_per_day,
                       marker_colors=[colors_dict[d] for d in steps_per_day.index])]) \
  .update_layout(title_text='Total Steps Daily') \
  .update_traces(textinfo='value', marker=dict(line=dict(color='black', width=3))).show()

# ========================
# Model Part with Evaluation
# ========================

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from keras.callbacks import EarlyStopping

# Load and preprocess
data = pd.read_csv('https://raw.githubusercontent.com/4522Dev/Dataset/refs/heads/main/dailyActivity_merged.csv')
data['ActivityDate'] = pd.to_datetime(data['ActivityDate'])
data.set_index('ActivityDate', inplace=True)

features = ['TotalSteps', 'TotalDistance', 'Calories']
data = data[features].fillna(0)

# Normalize
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), columns=features)

# Sequence generator
def create_sequences(data, time_steps=7):
    sequences, labels = [], []
    for i in range(len(data) - time_steps):
        sequences.append(data[i:(i + time_steps)])
        labels.append(data[i + time_steps])
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data.values)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape for LSTM input
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(features)))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], len(features)))

# Build model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], len(features))))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(50))
model.add(Dense(len(features)))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate
loss = model.evaluate(X_test, y_test)
print(f"\nTest Loss (from model.evaluate): {loss:.4f}")

# Predictions
predictions = model.predict(X_test)
predictions_inverse_scaled = scaler.inverse_transform(predictions)
y_test_inverse_scaled = scaler.inverse_transform(y_test)

# DataFrame of predictions
predictions_df = pd.DataFrame(predictions_inverse_scaled, columns=features)
print("\nSample Predictions:")
print(predictions_df.head())

# MSE metrics
mse_total_steps = mean_squared_error(y_test_inverse_scaled[:, 0], predictions_inverse_scaled[:, 0])
mse_total_distance = mean_squared_error(y_test_inverse_scaled[:, 1], predictions_inverse_scaled[:, 1])
mse_calories = mean_squared_error(y_test_inverse_scaled[:, 2], predictions_inverse_scaled[:, 2])

print("\nMean Squared Error:")
print(f"Total Steps     : {mse_total_steps:.4f}")
print(f"Total Distance  : {mse_total_distance:.4f}")
print(f"Calories Burned : {mse_calories:.4f}")

Id                          0
ActivityDate                0
TotalSteps                  0
TotalDistance               0
TrackerDistance             0
LoggedActivitiesDistance    0
VeryActiveDistance          0
ModeratelyActiveDistance    0
LightActiveDistance         0
SedentaryActiveDistance     0
VeryActiveMinutes           0
FairlyActiveMinutes         0
LightlyActiveMinutes        0
SedentaryMinutes            0
Calories                    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        940 non-null    int64  
 1   ActivityDate              940 non-null    object 
 2   TotalSteps                940 non-null    int64  
 3   TotalDistance             940 non-null    float64
 4   TrackerDistance           940 non-null    float64
 5   LoggedActivitiesDistance  940 non-null    

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.0519 - val_loss: 0.0201
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0152 - val_loss: 0.0139
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0117 - val_loss: 0.0132
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0112 - val_loss: 0.0127
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0109 - val_loss: 0.0126
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0083 - val_loss: 0.0126
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0100 - val_loss: 0.0126
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0093 - val_loss: 0.0126
Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0090 - val_loss: 0.0126
Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0090 - val_loss: 0.0125
Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0104 - val_loss: 0.0130
Epoch 12/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0089 - 